In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-65c6fbe3-9da8-df49-9f7e-3f3bb55ae785)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Graduation_Project/CheXpert-v1.0-small.zip > /dev/null

In [3]:
!pip install timm > /dev/null

In [5]:
import cv2
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

import torch 
from torch import nn
from torchsummary import summary

import timm
from torch.optim import Adam, lr_scheduler

from myGen import MyGen

from sklearn.metrics import roc_auc_score

In [6]:
train = pd.read_csv('/content/CheXpert-v1.0-small/train.csv')

In [7]:
train.loc[:, train.columns[5:]] = train.loc[:, train.columns[5:]].fillna(0)

zeros = ['Consolidation']
train.loc[:, zeros] = train.loc[:, zeros].replace({-1:0})
ones = ['Edema', 'Atelectasis', 'Cardiomegaly', 'Pleural Effusion']
train.loc[:, ones] = train.loc[:, ones].replace({-1:1})

train = train[['Path', 'Frontal/Lateral', 'Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']].copy() 

In [8]:
train.to_csv('/content/CheXpert-v1.0-small/train_edited.csv', index=False)

In [9]:
save_weights_path = '/content/drive/MyDrive/Graduation_Project/xception_new.pth' #edit
load_weights_path = '/content/drive/MyDrive/Graduation_Project/xception_new.pth' #edit

In [10]:
model = timm.create_model('xception', num_classes=5, pretrained=True) #edit
#model = model.cuda()

#summary(model, (3, 224, 224), batch_size=64, device='cuda')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth


In [12]:
model.load_state_dict(torch.load(load_weights_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [13]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
root = '/content/'

trainGen = MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/train_edited.csv', 
                 image_size=224, #edit
                 frontal=True)

valGen =  MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/valid.csv', 
                 image_size=224,  #edit
                 frontal=True)

trainloader =  torch.utils.data.DataLoader(trainGen, batch_size=64, shuffle=True)
testloader =  torch.utils.data.DataLoader(valGen, batch_size=64, shuffle=False)

# define loss & optimizer
Loss = nn.BCELoss()

optimizer = Adam(model.parameters(), lr=1e-3)

scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Decay LR by a factor of 0.5 every 5 epochs

#model.load_state_dict(torch.load(load_weights_path))

# training
const_threshold = 5

val_auc_mean = 0
best_val_auc = 0.8887
const_cnt = 0

for epoch in range(4):

    for idx, data in enumerate(trainloader):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      y_pred = torch.sigmoid(y_pred)
      loss = Loss(y_pred, train_labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      # validation  
      if idx % 400 == 0:
         model.eval()
         with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for data in testloader:
                  test_data, test_labels = data
                  test_data = test_data.cuda()
                  y_pred = model(test_data)
                  y_pred = torch.sigmoid(y_pred)
                  test_pred.append(y_pred.cpu().detach().numpy())            
                  test_true.append(test_labels.numpy())
            
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc_mean =  roc_auc_score(test_true, test_pred) 
              model.train()

              if best_val_auc < val_auc_mean:
                  const_cnt = 0
                  scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
                  best_val_auc = val_auc_mean
                  torch.save(model.state_dict(), save_weights_path)
              else:
                  const_cnt += 1
                  scheduler.step()            

              if (const_cnt > const_threshold):
                  const_cnt = 0 
                  scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
                  model.load_state_dict(torch.load(save_weights_path))
                               
              lr = get_lr(optimizer)

              print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f, lr=%f' %(epoch, idx, val_auc_mean, best_val_auc, lr))

Epoch=0, BatchID=0, Val_AUC=0.8912, Best_Val_AUC=0.8912, lr=0.001000
Epoch=0, BatchID=400, Val_AUC=0.8473, Best_Val_AUC=0.8912, lr=0.001000
Epoch=0, BatchID=800, Val_AUC=0.8693, Best_Val_AUC=0.8912, lr=0.001000
Epoch=0, BatchID=1200, Val_AUC=0.8833, Best_Val_AUC=0.8912, lr=0.001000
Epoch=0, BatchID=1600, Val_AUC=0.8759, Best_Val_AUC=0.8912, lr=0.001000
Epoch=0, BatchID=2000, Val_AUC=0.8672, Best_Val_AUC=0.8912, lr=0.000500
Epoch=0, BatchID=2400, Val_AUC=0.8710, Best_Val_AUC=0.8912, lr=0.000500
Epoch=0, BatchID=2800, Val_AUC=0.8915, Best_Val_AUC=0.8915, lr=0.000500
Epoch=1, BatchID=0, Val_AUC=0.8841, Best_Val_AUC=0.8915, lr=0.000500
Epoch=1, BatchID=400, Val_AUC=0.8715, Best_Val_AUC=0.8915, lr=0.000500
Epoch=1, BatchID=800, Val_AUC=0.8715, Best_Val_AUC=0.8915, lr=0.000500
Epoch=1, BatchID=1200, Val_AUC=0.8928, Best_Val_AUC=0.8928, lr=0.000500
Epoch=1, BatchID=1600, Val_AUC=0.8844, Best_Val_AUC=0.8928, lr=0.000500
Epoch=1, BatchID=2000, Val_AUC=0.8857, Best_Val_AUC=0.8928, lr=0.000500
Ep

In [14]:
root = '/content/'
testGen =  MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/valid.csv', 
                 image_size=224, 
                 frontal=True)

testloader =  torch.utils.data.DataLoader(testGen, batch_size=1, shuffle=False)

In [16]:
#model.load_state_dict(torch.load(save_weights_path))

In [20]:
model.eval()
with torch.no_grad():    
    test_pred = []
    test_true = [] 
    for data in testloader:
        test_data, test_labels = data
        test_data = test_data #.cuda()
        y_pred = model(test_data)
        y_pred = torch.sigmoid(y_pred)
        test_pred.append(y_pred.cpu().detach().numpy())
        test_true.append(test_labels.numpy())
  
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)

In [21]:
labels = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']

results = pd.DataFrame(index=labels)


scores = []
for i in range(5):
    score = roc_auc_score(test_true[:, i], test_pred[:, i])
    scores.append(score)
    
results['AUC'] = scores

In [22]:
results

,AUC
Cardiomegaly,0.816734
Edema,0.944643
Consolidation,0.912132
Atelectasis,0.847769
Pleural Effusion,0.942595


In [23]:
results['AUC'].mean()

0.8927745718475059